# Part 1 — Multi-Layer Perceptron (Age Regression)

| Name            | Student ID | Email                                      |
|-----------------|------------|--------------------------------------------|
| Valeria Avino   | 1905974    | avino.1905974@studenti.uniroma1.it         |
| Marta Lombardi  | 2156537    | lombardi.2156537@studenti.uniroma1.it      |


This notebook presents the implementation of a custom Multi-Layer Perceptron (MLP) for a **regression task** using the dataset `AGE REGRESSION.csv`. The goal is to minimize the **L2 regularized loss function** using manually derived gradients and a numerical optimizer provided by `scipy.optimize`.

The neural network:
- Uses **at least two hidden layers**
- Applies **L2 regularization** to the weight matrices
- Supports **sigmoid** or **tanh** activation functions
- Is optimized via **L-BFGS-B**, without relying on automatic differentiation libraries
- Evaluates performance with the **Mean Absolute Percentage Error (MAPE)**
- Selects hyperparameters using **k-fold cross-validation**

This notebook is structured to:
1. Preprocess the dataset
2. Define and optimize the MLP model from scratch
3. Perform model selection using cross-validation
4. Report training, validation, and test results



In [1]:
# importing all necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import make_scorer 
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import time

# used in the py file, here for explanability
from typing import Callable, Tuple, List, Dict, Any
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.exceptions import NotFittedError


# importing auxiliary functions from Functions.py file
from Functions_11_Avino_Lombardi import (
    forward, backward,
    g1, dg1_dx, g2, dg2_dx,
    mse_loss, mape, xavier_normal_init,
    initialize_parameters, unroll_params, roll_params,
    check_gradients_with_central_differences,
    objective_function, final_report_metrics, myMLPRegressor
)

### Data loading and splitting
After loading the data in our environment, we randomly partition it into training ( $80\% $) and testing ( $20\%$ ) sets. This separation is fundamental to evaluate the generalization capability of our trained model on unseen (test) data. 

In [2]:
df = pd.read_csv('/Users/Val/Documents/GitHub/OMDS-Project/dataset/AGE_PREDICTION.csv')

# Separate features (X_full) and target (y_full) from the entire dataset
feature_columns = [f'feat_{i}' for i in range(1, 33)]
X_full = df[feature_columns].values # Shape will be [N, D]
y_full = df['gt'].values.reshape(-1, 1) # Shape will be [N, 1]


print(f"Initial X shape: {X_full.shape}")
print(f"Initial y shape: {y_full.shape}")

# Get total number of samples
N_samples_full = X_full.shape[0]
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=1234)

X_train = X_train.T # Transpose to [D, Ntrain]
y_train = y_train.T # Transpose to [1, Ntrain]
X_test = X_test.T # Transpose to [D, N-Ntrain]
y_test = y_test.T # Transpose to [1, N-Ntrain]

# Determine D_input (number of input features) - from training data to check shapes
D_input = X_train.shape[0]
y_output_dim = y_train.shape[0]
print(f"D_input (number of input features): {D_input}")
print(f"y_output_dim (number of output dimensions): {y_output_dim}\n")
print(f"X_train shape after split: {X_train.shape}") # (N_features, N_train)
print(f"y_train shape after split: {y_train.shape}") # (1, N_train)

Initial X shape: (20475, 32)
Initial y shape: (20475, 1)
D_input (number of input features): 32
y_output_dim (number of output dimensions): 1

X_train shape after split: (32, 16380)
y_train shape after split: (1, 16380)


### Feature standardization

Before applying any optimization routine, we normalize our data
For each feature $x_i$:
$$x_i^{\text{normalized}}=\frac{x_i-\mu_i}{\sigma_i}$$

where $\mu_i$ and $\sigma_i$ are the **mean** and **standard deviation** of the $i^{th}$ feature, computed over the **training set**, since at this stage we do not have access to test information. The same transformation is then applied to test data.

Standardization ensures __all features contribute equally__ to the loss landscape, thus to the gradient updates, prevents issues like vanishing or exploding gradients due to varying feature scales, and accelerates the convergence of our L-BFGS-B optimizer.

In [3]:
# manual, but can be done with "StandardScaler"
# only computed from train data
mu = X_train.mean(axis=1, keepdims=True)
sigma = X_train.std(axis=1, keepdims=True)

# Handle cases where standard deviation might be zero (constant feature)
sigma[sigma == 0] = 1e-8 

# Apply the transformation to the TRAINING DATA
X_train_normalized = (X_train - mu) / sigma

# Apply the *SAME* transformation (using mu and sigma from training) to the TEST DATA
X_test_normalized = (X_test - mu) / sigma

Let's check if the gradient was correctly computed by the __Central differences__ approximation:
$$f'(x) \approx \frac{f(x+\Delta x)-f(x-\Delta x)}{2\Delta x}$$


In [4]:
print("\nPerforming gradient check...")

# subset of training data for gradient check 
num_samples_for_check = min(1000, X_train_normalized.shape[1])
X_check_subset = X_train_normalized[:, :num_samples_for_check]
y_check_subset = y_train[:, :num_samples_for_check]

# toy set of hyperparameters for the check
check_L = 3
check_neurons_config = [5, 5]
check_activation_func = g2 # also checked with g1 and different reg factors
check_activation_prime = dg2_dx
check_reg_factor = 0.001

# initialize parameters
np.random.seed( 1234 )
W_check_init, b_check_init, v_check_init = initialize_parameters(
    D_input, check_neurons_config, y_output_dim, check_reg_factor
)
initial_flat_params_for_check = unroll_params(W_check_init, b_check_init, v_check_init)

W_shapes_for_check = [W.shape for W in W_check_init]
b_shapes_for_check = [b.shape for b in b_check_init]
v_shape_for_check = v_check_init.shape

# gradient check function
check_gradients_with_central_differences(
    initial_flat_params_for_check,
    X_check_subset, y_check_subset,
    W_shapes_for_check, b_shapes_for_check, v_shape_for_check,
    check_activation_func, check_activation_prime,
    check_reg_factor, check_L,
    objective_function
)

print("Gradient check finished.")



Performing gradient check...

--- Gradient Check (Central Differences) ---
Checking 200 parameters...
Analytical loss at initial point: 1799.840240

Gradient check PASSED! Norm of difference: 1.566936e-06
------------------------------------------
Gradient check finished.


## Custom MLP Regressor Class

In `Functions_11_Avino_lombardi.py` we define a custom multi-layer perceptron (MLP) regressor class that integrates with the `scikit-learn` API by subclassing `BaseEstimator` (for hyperparameter handling and GridSearchCV support) and `RegressorMixin` (to behave like a standard sklearn regressor). It allows easy use of cross-validation and hyperparameter tuning via `GridSearchCV`.

The class implements:

- **Manual forward and backward propagation** (no autograd),
- **L2-regularized loss function** (objective),
- **Training via `scipy.optimize.minimize` with L-BFGS-B**, using analytical gradients,
- **Support for different activation functions** (tanh or sigmoid),
- **Hyperparameters** such as number of layers, neurons per layer and regularization strength.

It defines `.fit()` and `.predict()` methods following the sklearn standard, and it prints training loss periodically when enabled. This class is the core object used in the training and evaluation workflow that follows.

## K-Fold Cross Validation + Grid Search
 
To identify the optimal MLP architecture and hyperparameters, we perform **GridSearchCV** using **5-fold cross-validation**. 

#### Cross-Validation Strategy

We use `KFold` with `k = 5`, meaning the training data is split into 5 disjoint subsets. For each fold:
- The model is trained on 4 folds.
- It is validated on the remaining 1 fold.

This process repeats 5 times so that every fold serves once as validation.

#### Evaluation Metric: MAPE

We define a **custom MAPE scorer** (Mean Absolute Percentage Error) using `mape` function form our py file and `make_scorer` from `sklearn.metrics`, and negate it so that `GridSearchCV` treats **lower MAPE as better** (since it maximizes the score).

#### Grid Search Space

We define a grid of hyperparameter combinations over:
- **Total number of layers** $L \in \{3, 4, 5\}$ (implying 2–4 hidden layers)
- **Hidden layer sizes**, e.g., `[32, 16]`, `[32, 16, 32]`
- **Activation function**: `'g1'` (tanh) or `'g2'` (sigmoid)
- **L2 regularization factor**: $\lambda \in \{0.001, 0.01\}$

Only valid combinations are retained (i.e., where the number of hidden layers matches the length of the neuron configuration).

#### Execution

We pass our custom `myMLPRegressor` estimator to `GridSearchCV`, along with:
- the filtered hyperparameter grid,
- the `KFold` splitter,
- the custom `MAPE` scorer,
- `n_jobs=-1` for parallelism.

The grid search returns the combination with the **lowest average validation MAPE** across all 5 folds, which is then used to retrain the final model on the full training set.


In [5]:
# Custom MAPE Scorer for GridSearchCV
# best = highest score --> negative mape 
mape_scorer = make_scorer(lambda y_true, y_pred: -mape(y_true.reshape(1,-1), y_pred.reshape(1,-1)), greater_is_better=True)

# X_train_normalized and y_train are now (D, N) and (1, N) 
# For scikit-learn, they need to be (N, D) and (N, 1).
X_train_sklearn = X_train_normalized.T
y_train_sklearn = y_train.T

X_test_sklearn = X_test_normalized.T
y_test_sklearn = y_test.T 

print(f"X_train_sklearn shape (for sklearn): {X_train_sklearn.shape}")
print(f"y_train_sklearn shape (for sklearn): {y_train_sklearn.shape}\n")


# Hyperparameter Search Space for GridSearchCV
param_grid_full = {
    'num_layers': [3, 4, 5], # Total no. of layers
    'num_neurons': [
        [32, 16], [32, 32], [64, 32],
        [16, 16, 16], [32, 16, 32], [32, 32, 32],
        [16, 8, 16, 8], [32, 16, 32, 16] 
    ],
    'activation_func_name': ['g1', 'g2'], 
    'regularization_factor': [0.001, 0.01],
}

# Keep only VALID combinations
filtered_param_grid = []
for L_val in param_grid_full['num_layers']:
    # Number of hidden layers is L_val - 1 (L is total layers, including output)
    expected_hidden_layers = L_val - 1
    for N_list_val in param_grid_full['num_neurons']:
        if len(N_list_val) == expected_hidden_layers:
            for act_name_val in param_grid_full['activation_func_name']:
                for reg_f_val in param_grid_full['regularization_factor']:
                    filtered_param_grid.append({
                        'num_layers': [L_val],
                        'num_neurons': [N_list_val],
                        'activation_func_name': [act_name_val],
                        'regularization_factor': [reg_f_val],
                    })

# Instantiate the MLP Regressor for cv, with a reduced maxiter for speed
mlp_estimator = myMLPRegressor(D_input=D_input, y_output_dim=y_output_dim, max_iter=500, print_callback_loss=False, random_state=1234) # no loss prints here

# Setup KFold for cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

print("Starting GridSearchCV based Hyperparameter Tuning...\n")

# Set up GridSearchCV 
grid_search = GridSearchCV(
    estimator=mlp_estimator,
    param_grid=filtered_param_grid, 
    cv=kf,
    scoring=mape_scorer, 
    verbose=2, 
    n_jobs=-1,
    error_score=np.nan ,
    refit=True # for initial loss calculation
)

# Run the grid search 
start_time_grid_search = time.time()
grid_search.fit(X_train_sklearn, y_train_sklearn)
end_time_grid_search = time.time()
grid_search_time = end_time_grid_search - start_time_grid_search

print("\nGridSearchCV completed.")
print(f"GridSearchCV took {grid_search_time:.2f} seconds to complete.")

# Results = Best hyperparameters
best_overall_params = grid_search.best_params_
best_overall_mape_score = -grid_search.best_score_ # return to mape (positive)

X_train_sklearn shape (for sklearn): (16380, 32)
y_train_sklearn shape (for sklearn): (16380, 1)

Starting GridSearchCV based Hyperparameter Tuning...

Fitting 5 folds for each of 32 candidates, totalling 160 fits

GridSearchCV completed.
GridSearchCV took 1510.16 seconds to complete.


### Retraining the optimal model and Testing

After identifying the best-performing hyperparameter combination via cross-validation, we now **retrain the model** using these optimal settings on the **entire training dataset**. This allows the model to fully leverage all available training data for learning.

Once training is complete, we evaluate the model’s **generalization performance** by predicting on the **held-out test set**, which has remained completely unseen throughout training and validation. This final evaluation provides a realistic estimate of the model’s predictive accuracy on new, real-world data.


In [9]:
print("\n--- Retraining Optimal Model on Full TRAINING Dataset (max_iter=5000) ---")

# Extract final parameters from best_overall_params
final_L = best_overall_params['num_layers']
final_neurons_config = best_overall_params['num_neurons']
final_activation_name = best_overall_params['activation_func_name']
final_reg_factor = best_overall_params['regularization_factor']
final_train_max_iter = 5000

final_activation_func = g1 if final_activation_name == 'g1' else g2
final_activation_prime = dg1_dx if final_activation_name == 'g1' else dg2_dx

print(f"\nFinal training uses Optimal Configuration:")
print(f"  Layers (L): {final_L}")
print(f"  Neurons per Layer: {final_neurons_config}")
print(f"  Activation Function: {final_activation_func.__name__}")
print(f"  Regularization Factor: {final_reg_factor}")
print(f"  Max Iterations for L-BFGS-B (Final Train): {final_train_max_iter}")


final_model = myMLPRegressor(
    D_input=D_input,
    y_output_dim=y_output_dim, 
    num_layers=final_L,
    num_neurons=final_neurons_config,
    activation_func_name=final_activation_name,
    regularization_factor=final_reg_factor,
    max_iter=final_train_max_iter,
    print_callback_loss=True, # callback printing to track mse loss behaviour
    random_state=1234
)

# Calculate Initial Training Error (Regularized MSE & MAPE) 

np.random.seed(1234)
W_final_init, b_final_init, v_final_init = initialize_parameters(
    D_input, final_neurons_config, y_output_dim, final_reg_factor
)
initial_flat_params_final_model = unroll_params(W_final_init, b_final_init, v_final_init)

W_shapes_final_model = [W.shape for W in W_final_init]
b_shapes_final_model = [b.shape for b in b_final_init] 
v_shape_final_model = v_final_init.shape

y_train_pred_initial_final_model, _, _ = forward(
    X_train_normalized, W_final_init, b_final_init, v_final_init, final_activation_func, final_L
)
initial_train_mape_final_model = mape(y_train, y_train_pred_initial_final_model)
initial_train_mse_reg_final_model, _ = objective_function(
    initial_flat_params_final_model,
    X_train_normalized, y_train, W_shapes_final_model, b_shapes_final_model, v_shape_final_model,
    final_activation_func, final_activation_prime, final_reg_factor, final_L
)

# Final Training on Full Training Data by calling .fit() on myMLPRegressor instance
start_time_final_train = time.time()
final_model.fit(X_train_sklearn, y_train_sklearn) 
end_time_final_train = time.time()

# predict on train to get train MAPE
y_train_pred_final = final_model.predict(X_train_sklearn)
final_train_mape = mape(y_train, y_train_pred_final)
final_train_mse_no_reg = mse_loss(y_train, y_train_pred_final)

# Extract results directly from final_model's attributes
final_training_time = end_time_final_train - start_time_final_train
final_training_iterations = final_model.n_iterations_
final_train_objective_value = final_model.final_objective_value_
result_final_train_message = final_model.optimization_message_ # message from the fitted model

print(f"\nFinal model training completed in {final_training_time:.2f} seconds over {final_training_iterations} iterations.")


--- Retraining Optimal Model on Full TRAINING Dataset (max_iter=5000) ---

Final training uses Optimal Configuration:
  Layers (L): 4
  Neurons per Layer: [16, 16, 16]
  Activation Function: g2
  Regularization Factor: 0.001
  Max Iterations for L-BFGS-B (Final Train): 5000
  Iteration 10: Non-regularized MSE Loss = 140.060655
  Iteration 20: Non-regularized MSE Loss = 97.209427
  Iteration 30: Non-regularized MSE Loss = 96.100711
  Iteration 40: Non-regularized MSE Loss = 95.665333
  Iteration 50: Non-regularized MSE Loss = 95.495086
  Iteration 60: Non-regularized MSE Loss = 95.175409
  Iteration 70: Non-regularized MSE Loss = 95.008286
  Iteration 80: Non-regularized MSE Loss = 94.797841
  Iteration 90: Non-regularized MSE Loss = 94.583161
  Iteration 100: Non-regularized MSE Loss = 94.505955
  Iteration 110: Non-regularized MSE Loss = 94.377219
  Iteration 120: Non-regularized MSE Loss = 94.269977
  Iteration 130: Non-regularized MSE Loss = 94.147587
  Iteration 140: Non-regulariz

In [7]:
y_test_pred_raw = final_model.predict(X_test_sklearn)
test_error_mape = mape(y_test, y_test_pred_raw.reshape(1,-1))
final_trained_flat_params = unroll_params(final_model.W_list_, final_model.b_list_, final_model.v_)
test_mse_no_reg = mse_loss(y_test, y_test_pred_raw)


test_error_mse_reg, _ = objective_function(
    final_trained_flat_params,
    X_test_normalized, y_test, # Use X_test_normalized and y_test (original D,N and 1,N)
    final_model.W_shapes_, final_model.b_shapes_, final_model.v_shape_,
    final_model.activation_func_, final_model.activation_prime_, final_model.regularization_factor, final_model.num_layers
)
print(f"  Test Error (MAPE): {test_error_mape:.4f}%")
print(f"  Test Error (MSE, regularized): {test_error_mse_reg:.4f}")
print(f"  Test Error (MSE, non-regularized): {test_mse_no_reg:.4f}")

  Test Error (MAPE): 20.0649%
  Test Error (MSE, regularized): 95.7957
  Test Error (MSE, non-regularized): 94.7284


In [21]:
import importlib
import Functions_11_Avino_Lombardi
importlib.reload(Functions_11_Avino_Lombardi)
from Functions_11_Avino_Lombardi import final_report_metrics


### Final Report Metrics

In [22]:
# auxiliary function to display metrics for report
final_report_metrics(
    final_L=final_L,
    final_neurons_config=final_neurons_config,
    final_activation_name=final_activation_name,
    final_reg_factor=final_reg_factor,
    final_train_max_iter=final_train_max_iter,
    final_training_iterations=final_training_iterations,
    initial_train_mse_reg_final_model=initial_train_mse_reg_final_model,
    final_train_objective_value=final_train_objective_value,
    initial_train_mape_final_model=initial_train_mape_final_model,
    final_train_mape=final_train_mape,
    best_overall_mape_score=best_overall_mape_score,
    test_error_mape=test_error_mape,
    test_error_mse_reg=test_error_mse_reg, 
    result_final_train_message = result_final_train_message,
    final_train_mse_no_reg= final_train_mse_no_reg , test_error_mse_no_reg = test_mse_no_reg, 
)


--- Comprehensive Performance Metrics for Final Report ---

1. Optimal Model Configuration:
  Non-linearity (Activation Function): g2
  Total Number of Layers (L): 4
  Neurons per Layer (Nl): [16, 16, 16]
  Regularization Factor (λ): 0.001

2. Optimization Routine Details (for Final Training):
  Optimization Routine: L-BFGS-B
  Max Number of Iterations Parameter: 5000
  Returned Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Number of Iterations Performed: 337
  Starting Value of Objective Function: 1.7427e+03
  Final Value of Objective Function: 9.4165e+01

3. Training Set Performance:
  Initial Training Error (MAPE): 3569.5107%
  Initial Training Error (MSE, regularized): 1742.7159
  Final Training Error (MAPE): 20.1776%
  Final Training Error (MSE, regularized): 94.1650
  Final Training Error (MSE, **non-regularized**): 93.0976

4. Validation Set Performance (Average from K-Fold CV):
  Average Validation Error (MAPE): 20.3638%

5. Test Set Performance:
  Final Test Erro